In [214]:
import pandas as pd
import numpy as np
import re

In [297]:
file_name = 'raw_india2.csv'
df = pd.read_csv(file_name).drop('keyword',axis=1)
df.head()

,created_at,id,full_text,source,retweet_count,favorite_count,hashtags,user_handle,user_screen_name,user_location,user_followers_count,user_verified
0,Mon Mar 08 08:16:08 +0000 2021,1368837927178887168,Called my ex house-help asking her to get her ...,"<a href=""http://twitter.com/download/iphone"" r...",0,0,NaN,Shalini,ShaliniSainath,"Mumbai, India",85,False
1,Mon Mar 08 08:15:42 +0000 2021,1368837820287086594,I am allergic and take steroids for the same. ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",0,0,NaN,Mukul Garga,mukul3762,Uttar Pradesh,1235,False
2,Mon Mar 08 08:13:31 +0000 2021,1368837268585078784,My 5cents’ thought: Vaccine researchers and ma...,"<a href=""http://twitter.com/download/iphone"" r...",0,0,COVID19,deetcho,deetcho,Lao People's Democratic Republ,16,False
3,Mon Mar 08 08:13:30 +0000 2021,1368837263774351360,Covid death rate 10 times higher in overweight...,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",0,0,NaN,Sunder Barange,sunder_barange,"Indore, India",299,False
4,Mon Mar 08 08:13:05 +0000 2021,1368837158992220160,Corona in the World: AstraZeneca’s Vaccine Sus...,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",0,0,NaN,todaymynews1,todaymynews1,Gandhinagar,355,False


In [298]:
original_length = len(df)

In [299]:
# Deleting tweets out of India
delete_index = []
for i in range(0, len(df)):
    if(isinstance(df.iloc[i]['user_location'],str) and ("Bangladesh" in df.iloc[i]['user_location'] or "Pakistan" in df.iloc[i]['user_location'] or "Nepal" in df.iloc[i]['user_location'])):
        delete_index.append(i)
        
df = df.drop(df.index[delete_index])

In [300]:
# Deleting karnataka references to prevent contamination of dataset
delete_index = []
for i in range(0, len(df)):
    if(isinstance(df.iloc[i]['full_text'],str) and ("Karnataka" in df.iloc[i]['full_text']) and ("karnataka" in df.iloc[i]['full_text'])):
        delete_index.append(i)
        
df = df.drop(df.index[delete_index])

In [301]:
# Deleting news channels
news_handles = ['ndtv', 'aajtak', 'ABPNews', 'TimesNow', 'ZeeNews',
                'CNNnews18','ZoomTV','BBCHindi','DDNewslive','ndtvindia',
                'abpnewshindi','indiatvnews','ZeeTV','news7tamil','CNBCTV18Live',
                'News18UP','CNBCTV18News','BBCIndia','republic','IndiaToday','Quint',
                'FirstPost','The Wire','AltNews','Sudharshan','news18dotcom','IMPharmaNews',
                'fortis_hospital','moneycontrolcom','ANI','todayindiain','TOIIndiaNews',
                'ETimesLifestyle','HindustanTimes','TelanganaToday','DeccanHerald',
                'MirrorNow','DDOdiaNews','IndiaAheadNews','OpIndia','Outlook Magazine',
                'OpIndia.com','Zee News English','IndSamachar News','DailyO','timesofindia',
                'htTweets','thelivemirror','OfficialNews230','dna','ETPanache','MumbaiMirror']

for i in range(0, len(df)):
#     If raw_india2 is used
    if(file_name == 'raw_india2.csv' and (isinstance(df.iloc[i]['user_handle'],str) and df.iloc[i]['user_handle'] in news_handles) or (isinstance(df.iloc[i]['user_screen_name'],str) and df.iloc[i]['user_screen_name'] in news_handles)):
        delete_index.append(i)
#   If raw_india is used
    if(file_name == 'raw_india.csv' and (isinstance(df.iloc[i]['user_screen_name'],str) and df.iloc[i]['user_screen_name'] in news_handles)):
        delete_index.append(i)
        
df = df.drop(df.index[delete_index])

In [302]:
#Convert everything to lower case
df['full_text'] = df['full_text'].apply(lambda x: str(x).lower())

#Remove twitter handlers
df['full_text'] = df['full_text'].apply(lambda x:re.sub('@[^\s]+','',str(x)))

#remove hashtags
df['full_text'] = df['full_text'].apply(lambda x:re.sub(r'\B#\S+','',str(x)))

# Remove URLS
df['full_text'] = df['full_text'].apply(lambda x:re.sub(r"http\S+", "", str(x)))

#remove all single characters
df['full_text'] = df['full_text'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', ' ', str(x)))

# Substituting multiple spaces with single space
df['full_text'] = df['full_text'].apply(lambda x:re.sub(r'\s+', ' ', str(x), flags=re.I))

#Removing whitespace before and after the tweet
df['full_text'] = df['full_text'].apply(lambda x: str(x).strip())

# Getting & symbol correctly
df['full_text'] = df['full_text'].apply(lambda x:re.sub('&amp;','&',str(x)))

#Removing emojis
# emoji_pattern = re.compile(
#         "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags=re.UNICODE)
# df['full_text'] = df['full_text'].apply(lambda x: emoji_pattern.sub(r'', str(x)))

In [303]:
unique_tweets = {}
for i in range(0,len(df)):
    if df.iloc[i]['full_text'] in unique_tweets:
        if ((df.iloc[unique_tweets[df.iloc[i]['full_text']]]['retweet_count'] + df.iloc[unique_tweets[df.iloc[i]['full_text']]]['favorite_count']) < (df.iloc[i]['retweet_count'] + df.iloc[i]['favorite_count'])):
            unique_tweets[df.iloc[i]['full_text']] = i
    else:
        unique_tweets[df.iloc[i]['full_text']] = i

df = df.take(list(unique_tweets.values()))

In [304]:
if file_name == 'raw_india2.csv':
    df.to_csv('refined_india_tweets2.csv', index=False)
elif file_name == 'raw_india.csv':
    df.to_csv('refined_india_tweets.csv', index=False)

In [305]:
df.iloc[0]['full_text']

'called my ex house-help asking her to get her parents & elderly vaccinated & she goes “we are already taking the ayurvedic tablets didi” 🙆🏼\u200d♀️🙆🏼\u200d♀️🙆🏼\u200d♀️ took me lot of time explaining what vaccine is & why should they get it asap. vaccine to aa gaya ab sujh-boojh bhi aa jaye 🙏'

In [306]:
original_part1 = pd.read_csv('raw_india.csv')
original_part2 = pd.read_csv('raw_india2.csv')
original_length = len(original_part1) + len(original_part2)

refined_part1 = pd.read_csv('refined_india_tweets.csv')
refined_part2 = pd.read_csv('refined_india_tweets2.csv')
refined_length = len(refined_part1) + len(refined_part2)

print('Original total: ' + str(original_length) + '\nNew Length: ' + str(refined_length) + '\nTweets deleted: ' + str(original_length-refined_length))

Original total: 37909
New Length: 26147
Tweets deleted: 11762


In [307]:
comb_df = pd.concat([refined_part1, refined_part2])

In [308]:
unique_tweets = {}
for i in range(0,len(comb_df)):
    if comb_df.iloc[i]['full_text'] in unique_tweets:
        if ((comb_df.iloc[unique_tweets[comb_df.iloc[i]['full_text']]]['retweet_count'] + comb_df.iloc[unique_tweets[comb_df.iloc[i]['full_text']]]['favorite_count']) < (comb_df.iloc[i]['retweet_count'] + comb_df.iloc[i]['favorite_count'])):
            unique_tweets[comb_df.iloc[i]['full_text']] = i
    else:
        unique_tweets[comb_df.iloc[i]['full_text']] = i

comb_df = comb_df.take(list(unique_tweets.values()))

In [309]:
print(len(comb_df))
comb_df.to_csv('combined_india_tweets.csv',index=False)

21571


In [ ]:
comb_df = comb_df.reset_index()

In [317]:
print(comb_df[0:138]['full_text'])

0                              prevent spread, stay safe
1      healthcare professionals in the uk have moral ...
2      my parents are vaccinated with first dose of c...
3      it’s not important news that modi jee took cor...
4      two mayo staffers tested positive after 3 week...
                             ...                        
133    raksha mantri shri rajnath singh received the ...
134    at time when assembly elections are going on, ...
135    “choose vaccine over virus”, says dr ravinder ...
136    “unrealistic we’re going to finish this virus ...
137    maran anna chi , whether he takes vaccine now ...
Name: full_text, Length: 138, dtype: object
